In [0]:
%run ./_common

In [0]:
# set variables in python and sql
config_values = [
    ("catalog", "acme_corp"),
    ("schema", "retail"),
    ("internal_share_name", "internal_retail"),
    ("internal_recipient_name", "west_division"),
    ("external_share_name", "partner_retail"),
    ("external_recipient_name", "partner_co"),
]
DA = create_da_objects(config_values)

In [0]:
# create catalog if not exists
create_catalog = f'''
CREATE CATALOG IF NOT EXISTS {DA.catalog}
COMMENT 'Provider catalog for Delta Sharing demo'
'''
spark.sql(create_catalog)

In [0]:
# create schema if not exists
create_schema = f'''
CREATE SCHEMA IF NOT EXISTS {DA.catalog}.{DA.schema}
COMMENT 'Schema containing sales and customer data for sharing'
'''
spark.sql(create_schema)

In [0]:
%sql
USE CATALOG IDENTIFIER(DA.catalog);
USE SCHEMA IDENTIFIER(DA.schema);

In [0]:
%sql
-- Create customers table
CREATE OR REPLACE TABLE customers (
  customer_id INT,
  customer_name STRING,
  email STRING,
  country STRING,
  signup_date DATE,
  customer_segment STRING
)
COMMENT 'Customer master data';

-- Insert sample data
INSERT INTO customers VALUES
  (1, 'Acme Corp', 'contact@acme.com', 'USA', '2023-01-15', 'Enterprise'),
  (2, 'TechStart Inc', 'info@techstart.com', 'Canada', '2023-02-20', 'SMB'),
  (3, 'Global Solutions', 'hello@globalsol.com', 'UK', '2023-03-10', 'Enterprise'),
  (4, 'Innovate Ltd', 'contact@innovate.com', 'Australia', '2023-04-05', 'Mid-Market'),
  (5, 'Data Dynamics', 'team@datadyn.com', 'Germany', '2023-05-12', 'Enterprise'),
  (6, 'Cloud First', 'info@cloudfirst.com', 'USA', '2023-06-18', 'SMB'),
  (7, 'NextGen Analytics', 'contact@nextgen.com', 'Singapore', '2023-07-22', 'Mid-Market'),
  (8, 'Digital Transform', 'hello@digitrans.com', 'France', '2023-08-30', 'Enterprise');

In [0]:
%sql
-- Create sales transactions table
CREATE OR REPLACE TABLE sales_transactions (
  transaction_id INT,
  customer_id INT,
  product_name STRING,
  quantity INT,
  unit_price DECIMAL(10,2),
  total_amount DECIMAL(10,2),
  transaction_date DATE,
  region STRING
)
COMMENT 'Sales transaction data';

-- Insert sample sales data
INSERT INTO sales_transactions VALUES
  (101, 1, 'Data Platform License', 5, 1000.00, 5000.00, '2024-01-15', 'North America'),
  (102, 2, 'Analytics Suite', 2, 500.00, 1000.00, '2024-01-20', 'North America'),
  (103, 3, 'Data Platform License', 10, 1000.00, 10000.00, '2024-02-05', 'EMEA'),
  (104, 1, 'Storage Add-on', 3, 200.00, 600.00, '2024-02-10', 'North America'),
  (105, 4, 'Analytics Suite', 5, 500.00, 2500.00, '2024-02-15', 'APAC'),
  (106, 5, 'Data Platform License', 15, 1000.00, 15000.00, '2024-03-01', 'EMEA'),
  (107, 3, 'Storage Add-on', 8, 200.00, 1600.00, '2024-03-10', 'EMEA'),
  (108, 6, 'Analytics Suite', 1, 500.00, 500.00, '2024-03-15', 'North America'),
  (109, 7, 'Data Platform License', 7, 1000.00, 7000.00, '2024-04-01', 'APAC'),
  (110, 8, 'Storage Add-on', 10, 200.00, 2000.00, '2024-04-05', 'EMEA');

In [0]:
display_config_values(config_values)